In [29]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines.common.policies import MlpPolicy as PPO_MLP
from stable_baselines.deepq.policies import MlpPolicy as DQN_MLP
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import DQN
from environment.Stock_trading_env import Stock_trading
import altair as alt
# from experiment.visualization import visualize

In [30]:
import warnings
warnings.filterwarnings("ignore")

In [31]:
stock_list = ['sh.000001','sz.000002','sz.002371','sh.600004','sh.600030','sh.600036','sh.600276','sh.600519','sh.600900','sh.601318']

In [32]:
def find_file(path, name):
    for root, dirs, files in os.walk(path):
        for fname in files:
            if name in fname:
                return os.path.join(root, fname)

In [33]:
def stock_trade(stock_file):
    daily_wealth = []
    history = []
    df = pd.read_csv(stock_file)
    df = df.sort_values('date')

    env = DummyVecEnv([lambda: Stock_trading(df)])

    model = PPO2(PPO_MLP, env, verbose=0, tensorboard_log='.\\log')
    model.learn(total_timesteps=int(1e4))

    df_test = pd.read_csv(stock_file.replace('train', 'test'))

    env = DummyVecEnv([lambda: Stock_trading(df_test)])
    obs = env.reset()
    for i in range(len(df_test) - 1):
        action, _states = model.predict(obs)
#         print('ACTION:',action)
        obs, rewards, done, info = env.step(action)
        wealth, action_name = env.render()
        daily_wealth.append(wealth)
        history.append(action_name)
        if done:
            break
    return daily_wealth,history

In [34]:
def single_stock(stock_code):
    stock_file = find_file('.\\data\\train', str(stock_code))
    df_test = pd.read_csv(stock_file.replace('train', 'test'))

    daily_wealth,history = stock_trade(stock_file)
    vis_df=df_test[['date','close']]
    vis_df['close']=vis_df['close']/vis_df.loc[0,'close']
    vis_df['date'] = pd.to_datetime(vis_df['date'],infer_datetime_format=True)
    vis_df['wealth']=pd.DataFrame(([daily_wealth[0]]+daily_wealth)/daily_wealth[0])
    vis_df['action']=pd.DataFrame(['HOLD']+history)
    return vis_df

In [35]:
# def multi_stock():
#     group_result = []
#     stock_list = ['sh.000001','sz.000002','sz.002371','sh.600004','sh.600030','sh.600036','sh.600276','sh.600519','sh.600900','sh.601318']

#     for code in stock_list:
#         stock_file = find_file('.\\data\\train', str(code))
#         if stock_file:
#             try:
#                 profits = single_stock(stock_file)
#                 group_result.append(profits)
#             except Exception as e:
#                 print(e)

#     with open('multi_stock.pkl', 'wb') as f:
#         pickle.dump(group_result, f)

In [42]:
vis_df = single_stock('sh.600276')

----------------------------------------------------------------------------------------------------
Step: 1
Action:BUY 
Balance:375880.8244771719
Share(s): 13485 
Avg cost for held shares: 46.282475010962415 
Net wealth: 1004012.1244771718 (Max net wealth: 1004012.1244771718)
Profit: 4012.1244771718048
----------------------------------------------------------------------------------------------------
Step: 2
Action:BUY 
Balance:219059.81420366795
Share(s): 16851 
Avg cost for held shares: 46.34384818683355 
Net wealth: 1004147.904203668 (Max net wealth: 1004147.904203668)
Profit: 4147.904203668004
----------------------------------------------------------------------------------------------------
Step: 3
Action:BUY 
Balance:43.148769294057274
Share(s): 21588 
Avg cost for held shares: 46.320032019210025 
Net wealth: 992875.268769294 (Max net wealth: 1004147.904203668)
Profit: -7124.7312307059765
-----------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 96
Action:HOLD 
Balance:48.765091362409294
Share(s): 21116 
Avg cost for held shares: 54.048156771093986 
Net wealth: 1051414.4050913623 (Max net wealth: 1271231.9650913624)
Profit: 51414.40509136231
----------------------------------------------------------------------------------------------------
Step: 97
Action:HOLD 
Balance:48.765091362409294
Share(s): 21116 
Avg cost for held shares: 54.048156771093986 
Net wealth: 1071896.9250913623 (Max net wealth: 1271231.9650913624)
Profit: 71896.92509136233
----------------------------------------------------------------------------------------------------
Step: 98
Action:HOLD 
Balance:48.765091362409294
Share(s): 21116 
Avg cost for held shares: 54.048156771093986 
Net wealth: 1057538.0450913624 (Max net wealth: 1271231.9650913624)
Profit: 57538.04509136244
-------------------------------------------------------------------------------

Action:HOLD 
Balance:13.289990990801016
Share(s): 20550 
Avg cost for held shares: 49.784310556417964 
Net wealth: 1197667.289990991 (Max net wealth: 1271231.9650913624)
Profit: 197667.2899909909
----------------------------------------------------------------------------------------------------
Step: 171
Action:HOLD 
Balance:13.289990990801016
Share(s): 20550 
Avg cost for held shares: 49.784310556417964 
Net wealth: 1200749.789990991 (Max net wealth: 1271231.9650913624)
Profit: 200749.7899909909
----------------------------------------------------------------------------------------------------
Step: 172
Action:HOLD 
Balance:13.289990990801016
Share(s): 20550 
Avg cost for held shares: 49.784310556417964 
Net wealth: 1186775.789990991 (Max net wealth: 1271231.9650913624)
Profit: 186775.7899909909
----------------------------------------------------------------------------------------------------
Step: 173
Action:SELL 
Balance:525259.2733290786
Share(s): 11525 
Avg cost for held share

Avg cost for held shares: 74.09365665578773 
Net wealth: 1577228.1346873434 (Max net wealth: 1696020.5865525901)
Profit: 577228.1346873434
----------------------------------------------------------------------------------------------------
Step: 269
Action:SELL 
Balance:471767.53035407653
Share(s): 14851 
Avg cost for held shares: 75.30194116603445 
Net wealth: 1496635.0403540768 (Max net wealth: 1696020.5865525901)
Profit: 496635.0403540768
----------------------------------------------------------------------------------------------------
Step: 270
Action:BUY 
Balance:54.29945248714648
Share(s): 21870 
Avg cost for held shares: 72.70335432822895 
Net wealth: 1480434.5994524872 (Max net wealth: 1696020.5865525901)
Profit: 480434.5994524872
----------------------------------------------------------------------------------------------------
Step: 271
Action:HOLD 
Balance:54.29945248714648
Share(s): 21870 
Avg cost for held shares: 72.70335432822895 
Net wealth: 1511489.999452487 (Max ne

Avg cost for held shares: 82.43826065314312 
Net wealth: 1589278.0354472385 (Max net wealth: 2037839.0055571473)
Profit: 589278.0354472385
----------------------------------------------------------------------------------------------------
Step: 358
Action:HOLD 
Balance:6.785447238536435
Share(s): 20125 
Avg cost for held shares: 82.43826065314312 
Net wealth: 1602359.2854472385 (Max net wealth: 2037839.0055571473)
Profit: 602359.2854472385
----------------------------------------------------------------------------------------------------
Step: 359
Action:HOLD 
Balance:6.785447238536435
Share(s): 20125 
Avg cost for held shares: 82.43826065314312 
Net wealth: 1604975.5354472385 (Max net wealth: 2037839.0055571473)
Profit: 604975.5354472385
----------------------------------------------------------------------------------------------------
Step: 360
Action:HOLD 
Balance:6.785447238536435
Share(s): 20125 
Avg cost for held shares: 82.43826065314312 
Net wealth: 1538563.0354472385 (Max n

----------------------------------------------------------------------------------------------------
Step: 446
Action:HOLD 
Balance:39.15186240551441
Share(s): 21254 
Avg cost for held shares: 69.4829645267506 
Net wealth: 1436809.5518624054 (Max net wealth: 2037839.0055571473)
Profit: 436809.5518624054
----------------------------------------------------------------------------------------------------
Step: 447
Action:HOLD 
Balance:39.15186240551441
Share(s): 21254 
Avg cost for held shares: 69.4829645267506 
Net wealth: 1402590.6118624054 (Max net wealth: 2037839.0055571473)
Profit: 402590.61186240544
----------------------------------------------------------------------------------------------------
Step: 448
Action:HOLD 
Balance:39.15186240551441
Share(s): 21254 
Avg cost for held shares: 69.4829645267506 
Net wealth: 1390263.2918624054 (Max net wealth: 2037839.0055571473)
Profit: 390263.2918624054
------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
Step: 533
Action:HOLD 
Balance:1064206.4528273568
Share(s): 4930 
Avg cost for held shares: 83.69647608247578 
Net wealth: 1409602.2528273568 (Max net wealth: 2037839.0055571473)
Profit: 409602.2528273568
----------------------------------------------------------------------------------------------------
Step: 534
Action:BUY 
Balance:30.845625851769
Share(s): 20126 
Avg cost for held shares: 73.37768231581589 
Net wealth: 1439442.3656258518 (Max net wealth: 2037839.0055571473)
Profit: 439442.3656258518
----------------------------------------------------------------------------------------------------
Step: 535
Action:HOLD 
Balance:30.845625851769
Share(s): 20126 
Avg cost for held shares: 73.37768231581589 
Net wealth: 1491367.4456258516 (Max net wealth: 2037839.0055571473)
Profit: 491367.44562585163
--------------------------------------------------------------------------------------

Balance:4.9933013236150146
Share(s): 19799 
Avg cost for held shares: 66.07688838493092 
Net wealth: 1353662.6233013237 (Max net wealth: 2037839.0055571473)
Profit: 353662.62330132374
----------------------------------------------------------------------------------------------------
Step: 608
Action:HOLD 
Balance:4.9933013236150146
Share(s): 19799 
Avg cost for held shares: 66.07688838493092 
Net wealth: 1308916.8833013235 (Max net wealth: 2037839.0055571473)
Profit: 308916.8833013235
----------------------------------------------------------------------------------------------------
Step: 609
Action:HOLD 
Balance:4.9933013236150146
Share(s): 19799 
Avg cost for held shares: 66.07688838493092 
Net wealth: 1334457.5933013237 (Max net wealth: 2037839.0055571473)
Profit: 334457.5933013237
----------------------------------------------------------------------------------------------------
Step: 610
Action:HOLD 
Balance:4.9933013236150146
Share(s): 19799 
Avg cost for held shares: 66.07688

----------------------------------------------------------------------------------------------------
Step: 670
Action:HOLD 
Balance:5.172761915484443
Share(s): 18536 
Avg cost for held shares: 81.83138038269736 
Net wealth: 1489372.7727619153 (Max net wealth: 2037839.0055571473)
Profit: 489372.77276191534
----------------------------------------------------------------------------------------------------
Step: 671
Action:HOLD 
Balance:5.172761915484443
Share(s): 18536 
Avg cost for held shares: 81.83138038269736 
Net wealth: 1458603.0127619153 (Max net wealth: 2037839.0055571473)
Profit: 458603.01276191534
----------------------------------------------------------------------------------------------------
Step: 672
Action:HOLD 
Balance:5.172761915484443
Share(s): 18536 
Avg cost for held shares: 81.83138038269736 
Net wealth: 1538493.1727619155 (Max net wealth: 2037839.0055571473)
Profit: 538493.1727619155
--------------------------------------------------------------------------------

In [43]:
vis1=vis_df[['date','close','action']]
vis1['category']='close'
vis1.rename(columns={'close':'Net Worth'},inplace=True)

In [44]:
vis2=vis_df[['date','wealth']]
vis2['action']='HOLD'
vis2['category']='wealth'
vis2.rename(columns={'wealth':'Net Worth'},inplace=True)

In [45]:
new_vis=pd.concat((vis1,vis2),axis=0)

In [46]:
def visualize(df):
    line_df=df[['date','Net Worth','category']]
#     scale = alt.Scale(domain=(min(df['Net Worth']) - 0.2, max(df['Net Worth']) + 0.2),clamp=True)
    
    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')
    
    line = alt.Chart(line_df).mark_line(interpolate='basis').encode(x='date:T',y='Net Worth:Q',color='category:N')
    
    action_points = alt.Chart(df).transform_filter(alt.datum.action != 'HOLD').mark_point(filled=True).encode(x=alt.X('date:T', axis=alt.Axis(title='Date')),y=alt.Y('Net Worth', axis=alt.Axis(format='.4f', title='Net Worth')),color='action').interactive(bind_y=False)
    
    selectors = alt.Chart(line_df).mark_point().encode(x='date:T',opacity=alt.value(0),).add_selection(nearest)
    
    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))
    
    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(text=alt.condition(nearest, 'Net Worth:Q', alt.value(' ')))
    
    # Draw a rule at the location of the selection
    rules = alt.Chart(line_df).mark_rule(color='gray').encode(x='date:T',).transform_filter(nearest)
    
    chart = alt.layer(line,selectors, points,rules,text,action_points,title="Portfolio Actions").properties(height=300, width=800)
    
    return chart

In [47]:
chart=visualize(new_vis)
chart

alt.LayerChart(...)